In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import codecs
import pandas as pd
from pandas import json_normalize
import os
from pathlib import Path
        

In [ ]:
# Step 1 - Obtain website soup for the given league and season
url = 'https://understat.com/match/4406'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Step 2 - Identify the players data
roster_home = soup.find("div", class_ = 'roster roster-home'); roster_home
roster_away = soup.find("div", class_ = 'roster roster-away')

In [ ]:
rhp = roster_home.find_all('label', class_='player'); rhp
rap = roster_away.find_all('label',class_='player')

In [ ]:
print(rhp[0])
print(rhp[0].find('span',class_ = 'player-name').text)
print(rhp[0]['data-id'])

In [ ]:
schema = ['matchId','date','homeTeam','awayTeam','homeScore','awayScore','playerId','playerName','playerPosition','playerTime','playerSide']
df_squad = pd.DataFrame(columns = schema)

for matchId in range(1): # Guessing a maximum of 20.000 matches
    header = soup.find('div', class_ = 'header-wrapper') # Extracting the header of the page
    teams  = header.find_all('a',class_ = not 'link-icon'); # Extracting the name of the teams
    score = [header.find('span').text.split(' - ')[0][-1], header.find('span').text.split(' - ')[1][0]]
    breadcrumb = soup.find('ul',class_ = 'breadcrumb'); # Extracting date
    date = breadcrumb.find_all('li')[-1].text
    for player_data in rhp:
        temp = {}
        temp['matchId'] = matchId
        temp['date'] = date
        temp['homeTeam'] = teams[0].text
        temp['awayTeam'] = teams[1].text
        temp['homeScore'] = score[0]
        temp['awayScore'] = score[1]
        temp['playerId'] = player_data['data-id']
        temp['playerName'] = player_data.find('span',class_ = 'player-name').text
        temp['playerPosition'] = player_data.find('span',class_ = 'player-position').text
        temp['playerTime'] = player_data.find('span',class_ = 'player-time').text
        temp['playerSide'] = 'h'
        df_squad = df_squad.append(pd.DataFrame(temp,index = [0]),ignore_index = True)
    for player_data in rap:
        temp = {}
        temp['matchId'] = matchId
        temp['date'] = date
        temp['homeTeam'] = teams[0].text
        temp['awayTeam'] = teams[1].text
        temp['homeScore'] = score[0]
        temp['awayScore'] = score[1]
        temp['playerId'] = player_data['data-id']
        temp['playerName'] = player_data.find('span',class_ = 'player-name').text
        temp['playerPosition'] = player_data.find('span',class_ = 'player-position').text
        temp['playerTime'] = player_data.find('span',class_ = 'player-time').text
        temp['playerSide'] = 'a'
        df_squad = df_squad.append(pd.DataFrame(temp,index = [0]),ignore_index = True)


In [ ]:
df_squad

In [ ]:
class match:
    def __init__(id_, rhp, rap):
        matchId = id_
        df_posconf = pd.read_excel('config/positions_config.xlsx') # Load the positions config file
        macropositions = df_posconf['Macro position'].unique().tolist();
        

    def map_position(pos):
        # trim R and L from the end
        if pos[-1] == 'R':
            pos = pos.rsplit('R',1)[0]
        elif pos[-1] == 'L':
            pos = pos.rsplit('L',1)[0]
        # Define macro pos
        try:
            macro = df_posconf.loc[df_posconf['Abbreviation'] == pos,'Macro position']
        except KeyError:
            macro = df_posconf.loc[df_posconf['Abbreviation'] =='Default','Macro position']
        

In [ ]:
'RAB'.rsplit('R',1)

In [ ]:
header = soup.find('div', class_ = 'header-wrapper'); #print(header)
squads  = header.find_all('a',class_ = not 'link-icon');
breadcrumb = soup.find('ul',class_ = 'breadcrumb'); #print(breadcrumb)
print('Date: ', breadcrumb.find_all('li')[-1].text)
print('Home team: ', squads[0].text)
print('Away team: ', squads[1].text)
print('Result: ',header.find('span').text.split(' - ')[0][-1],'-',header.find('span').text.split(' - ')[1][0])

## Explore table data

In [ ]:
scripts = soup.find_all('script'); scripts

In [ ]:
for script in scripts:
    if 'var rostersData' in str(script):
        # I store that text, then trim off the string on the ends so that 
        # it's in a valid json format
        encoded_string = str(script)
        encoded_string  = encoded_string.split("JSON.parse('",1)[-1]
        #encoded_string = encoded_string.rsplit("=",1)[0]
        encoded_string = encoded_string.rsplit("');",1)[0]
        # Have it ignore the escape characters so it can decode the ascii 
        # and be able to use json.loads
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)
        # Due to the many levels, we create a list of dicts to be processed by json_normalize
        jsonObjH = [jsonObj['h'][ii] for ii in list(jsonObj['h'])]
        jsonObjA = [jsonObj['a'][ii] for ii in list(jsonObj['a'])]
        db_h = json_normalize(jsonObjH).set_index('id')
        db_a = json_normalize(jsonObjA).set_index('id')
        db = db_h.append(db_a)

In [ ]:
db

In [ ]:
jsonObj['a']

# Alternative solution

In [ ]:
df = pd.DataFrame()
for matchId in range(20000): # Guessing a maximum of 20.000 matches
    url = 'https://understat.com/match/'+str(matchId)
    response = requests.get(url)
    if response.status_code == 200:
        print('OK for ',matchId)
        soup = BeautifulSoup(response.text, 'html.parser')
        header = soup.find('div', class_ = 'header-wrapper') # Extracting the header of the page
        score = [header.find('span').text.split(' - ')[0][-1], header.find('span').text.split(' - ')[1][0]]
        breadcrumb = soup.find('ul',class_ = 'breadcrumb'); # Extracting date
        date = breadcrumb.find_all('li')[-1].text
        scripts = soup.find_all('script'); # Scripts with data
        for script in scripts:
            if 'var rostersData' in str(script):
                # I store that text, then trim off the string on the ends so that 
                # it's in a valid json format
                encoded_string = str(script)
                encoded_string  = encoded_string.split("JSON.parse('",1)[-1]
                #encoded_string = encoded_string.rsplit("=",1)[0]
                encoded_string = encoded_string.rsplit("');",1)[0]
                # Have it ignore the escape characters so it can decode the ascii 
                # and be able to use json.loads
                jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
                jsonObj = json.loads(jsonStr)
                # Due to the many levels, we create a list of dicts to be processed by json_normalize
                jsonObjH = [jsonObj['h'][ii] for ii in list(jsonObj['h'])]
                jsonObjA = [jsonObj['a'][ii] for ii in list(jsonObj['a'])]
                db_h = json_normalize(jsonObjH).set_index('id')
                db_a = json_normalize(jsonObjA).set_index('id')
                db = db_h.append(db_a)
                db['matchId'] = matchId
                db['date'] = date
                db['homeScore'] = score[0]
                db['awayScore'] = score[1]
                break
        df = df.append(db)
    else:
        print('Not OK for ',matchId)

In [ ]:
df

In [ ]:
url = 'https://understat.com/match/4406'
response = requests.get(url)

In [ ]:
response

In [ ]:
Path('test.csv').exists()

In [ ]:
import pandas as pd

In [ ]:
from DataPreprocessing import DataPreprocessing

In [ ]:
dataStream = DataPreprocessing(update = True)

In [ ]:
from DataPreprocessing import DataPreprocessing
dataStream = DataPreprocessing()